In [40]:
import numpy as np
import matplotlib.pyplot as plt
import ipywidgets as widgets
from scipy.interpolate import interp1d

# Given data
average_powers = np.array([906.7, 1240.2, 1542.7, 1849.2, 2166.0, 2497.9, 2868.1, 3261.5, 3627.5])
average_efficiencies = np.array([47.194462, 42.118347, 39.609324, 38.250711, 37.462641, 37.112756, 37.500376, 37.913485, 38.430896])

# Convert efficiencies to TH/W for interpolation (since the provided data is W/TH)
average_efficiencies = 1 / average_efficiencies

# Create an interpolation function based on the average efficiency data
efficiency_interpolator = interp1d(average_powers, average_efficiencies, fill_value="extrapolate")

# Define BTC mined based on TH
def btc_mined(total_th, k):
    return k * total_th

# Define the TH/s RT given power using the efficiency lookup
def ths_rt_given_power(power):
    efficiency = efficiency_interpolator(power)   
    return power * efficiency

# Update the adjusted net profit function to use the new TH/s RT calculation
def updated_net_profit(power, btc_price, electricity_rate, k, duration=1):
    ths_rt = ths_rt_given_power(power)
    revenue = btc_mined(ths_rt, k) * btc_price
    electricity_cost = (power / 1000) * electricity_rate * duration  # Convert power to kW and multiply by rate and duration
    return revenue - electricity_cost

# Update the plotting function to use the new net profit calculation
def updated_plot_net_profit(btc_price, electricity_rate):
    k = 0.0000000914  # Fixed k value
    powers = np.arange(200, 3601, 50)  # Power values from 200W to 3600W
    profits = [updated_net_profit(p, btc_price, electricity_rate, k) for p in powers]
    
    plt.figure(figsize=(12, 8))
    plt.plot(powers, profits, label='Net Profit', color='blue')
    
    # Check if all profits are negative
    if all(p <= 0 for p in profits):
        plt.axhline(0, color='red', linestyle='--', label='Not Profitable')
    else:
        plt.axvline(powers[np.argmax(profits)], color='red', linestyle='--', label=f'Optimal Power: {powers[np.argmax(profits)]}W')
    
    plt.title("Net Profit vs. Power in Watts")
    plt.xlabel("Power (W)")
    plt.ylabel("Net Profit ($/hr)")
    plt.legend()
    plt.grid(True)
    plt.tight_layout()
    plt.show()

# Interactive widgets
btc_price_slider = widgets.FloatSlider(value=27500, min=1000, max=60000, step=500, description='BTC Price:')
electricity_rate_slider = widgets.FloatSlider(value=0.03, min=0.01, max=0.50, step=0.001, description='Electricity Rate:', readout_format='.3f')

# Display the updated interactive plot
widgets.interactive(updated_plot_net_profit, btc_price=btc_price_slider, electricity_rate=electricity_rate_slider)


interactive(children=(FloatSlider(value=27500.0, description='BTC Price:', max=60000.0, min=1000.0, step=500.0…